In [1]:
import torch
import torch.nn.functional as F
import random

In [2]:
x = torch.FloatTensor([[[1,2,3], [1,2,3]], [[2,3,4], [3,4,5]]])
codebook_size = 2

In [40]:
vectors = x.flatten(end_dim=1)
unique_vectors = vectors.unique(dim=0)

random_idx = torch.randperm(vectors.size(dim=0))[:codebook_size]
kmeans_centroids = vectors[random_idx]

In [41]:
torch.sum(vectors**2, dim=0) + torch.sum(kmeans_centroids**2, dim=0)

tensor([25., 53., 93.])

In [42]:
torch.matmul(vectors, kmeans_centroids.t())

tensor([[14., 26.],
        [14., 26.],
        [20., 38.],
        [26., 50.]])

In [43]:
distances = torch.sum(vectors**2, dim=1, keepdim=True) + torch.sum(kmeans_centroids**2, dim=1) - 2.0 * torch.matmul(vectors, kmeans_centroids.t())

In [44]:
dists, indexes = torch.min(distances, dim=1)

In [45]:
vectors.shape

torch.Size([4, 3])

In [46]:
for i in range(codebook_size):
    cluster_indices = torch.nonzero(indexes == i).squeeze(1)
    if cluster_indices.shape[0] > 0:
        one_hot_eq = F.one_hot(cluster_indices, num_classes=vectors.shape[0]).float()
        cluster_vals = torch.matmul(one_hot_eq, vectors)
        average = cluster_vals.sum(dim=0) / cluster_indices.shape[0]
        
        kmeans_centroids[i] = average
    else:
        kmeans_centroids[i] = vectors[random.randint(0, len(vectors))]
    

In [39]:
kmeans_centroids

tensor([[1.3333, 2.3333, 3.3333],
        [3.0000, 4.0000, 5.0000]])